In [ ]:
import os
import json
import glob
import re
from src.wimbd_ import _load_dataset

HF_HOME = "/share/edc/home/antonis/datasets/huggingface"
save_path = os.makedirs(os.path.join(HF_HOME, 'wmt09_gens'), exist_ok=True)

base_pth = "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/models/experiment_6_logits_max_4/"
doc_res_files = glob.glob(os.path.join(base_pth, "**/doc_results.json"), recursive=True)


In [ ]:
pattern = re.compile(r'wmt\d{2}-(\w{2}-\w{2})')
language_pairs = [pattern.search(path).group(1) for path in doc_res_files if pattern.search(path)]
doc = json.load(open(doc_res_files[0]))

def find_lang(path):
    pattern = re.compile(r'wmt\d{2}-(\w{2}-\w{2})')
    return pattern.search(path).group(1)

wmt = _load_dataset('wmt')

In [ ]:
from tqdm import tqdm
from datasets import DatasetDict
from datasets import Dataset

def insert_gens_into_wmt(wmt, doc_res_files):
    for path in tqdm(doc_res_files, desc="Processing files"):
        wmt_model = wmt.copy()

        with open(path, 'r') as f:
            doc_res = json.load(f)

        model_name = path.split('/')[-5]
        task = path.split('/')[-3]
        lang = find_lang(path)
        lang1, lang2 = lang.split('-')
        total_rows = len(wmt_model[lang]['translation'])
        matched_rows = 0
        doc_res_task_dict = wmt_model[lang].to_dict()
        len_task_dict = len(doc_res_task_dict['translation'])
        unmatched_rows = [i for i in range(len_task_dict)]

        for doc in tqdm(doc_res[task], desc=f"Inserting generations for {task}", leave=False):
            src = doc['src']
            ref = doc['ref']
            gen = doc['result'][0]

            for idx in unmatched_rows:
                row = doc_res_task_dict['translation'][idx]
                # print(row)
                if row[lang1] != src and row[lang2] != ref:
                    continue
                else:
                    doc_res_task_dict['translation'][idx]['gen'] = gen
                    matched_rows += 1
                    unmatched_rows.remove(idx)
                    break
        
        if len(unmatched_rows) > 0:
            print(f"Could not match {len(unmatched_rows)} rows for {task}")

        wmt_model[lang] = Dataset.from_dict(doc_res_task_dict)
        dataset_dict = {lang: Dataset.from_dict({'translation': value['translation']}) for lang, value in wmt_model.items()}
        dataset_dict = DatasetDict(dataset_dict)
        # Now you can save it to disk
        dataset_dict.save_to_disk(os.path.join(HF_HOME, f'wmt09_gens_{model_name}'))
        print(f"Matched {matched_rows}/{total_rows} rows for {task}")

    return wmt


wmt = insert_gens_into_wmt(wmt, doc_res_files)




# dataset_dict = {lang: Dataset.from_dict({'translation': value['translation']}) for lang, value in wmt.items()}
# dataset_dict = DatasetDict(dataset_dict)

# Now you can save it to disk
# dataset_dict.save_to_disk(os.path.join(HF_HOME, 'wmt09_gens'))